In [1]:
import numpy as np
config = {
    # base info about dataset
    'train_num':44375,
    'test_num':21435,
    'test_num':21435,

    # paradim setting
    'sentence_len':8,
    'use_stopwords':True,

}
stoplist = set('a the this that these those of on in to for'.split())

gensim.models.keyedvectors.KeyedVectors

In [10]:
from qa_set import QASet

QA = QASet(part="train")

In [11]:
qst_vec,ans_vec,image_id = QA.__getitem__(1)

In [12]:
print("qst_vec_shape : {} , ans_vec_shape : {} , image_id : {}".format(qst_vec.shape,ans_vec.shape,image_id))

qst_vec_shape : (8, 100) , ans_vec_shape : (8, 100) , image_id : 393227
